In [8]:
# Import dependencies 
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy import inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

In [9]:
# Load CSV data 
pd.set_option('display.max_columns', None)
sold_homes_df = pd.read_csv('./Resources/sold_homes_combined_final.csv')
sold_homes_df

,#,LSC,EC,St#,Street Name,Abbr,Dir,Municipality,Community,List Price,Sold Price,Type,Style,Br,Additional,Wr,Fam,Kit,Garage Type,A/C,Heat,Contract Date,Sold Date,List Brokerage,Co op Brokerage,MLS#
0,1,Sld,NaN,38,Scarsdale,Crt,NaN,Brampton,Brampton W,"$599,000","$681,000",Att/Row/Tw,2-Storey,3,0,2,N,1,Built-,Cen,Gas,3/16/2021,3/23/2021,RE/MAX WEST,RE/MAX REALTY,W5153139
1,2,Sld,NaN,55,Corby,Cres,NaN,Brampton,Brampton W,"$599,900","$767,000",Semi-Detac,Bungalow,3,1,2,N,1,None,Cen,Oil,2/23/2021,2/26/2021,RE/MAX REAL,HOMELIFE/MIRA,W5125388
2,3,Sld,NaN,7,Manett,Cres,NaN,Brampton,Brampton W,"$699,000","$818,500",Att/Row/Tw,2-Storey,3,1,3,N,1,Attach,Cen,Gas,1/21/2021,1/26/2021,RE/MAX REAL,HOMELIFE/MIRA,W5091089
3,4,Sld,NaN,76,Fairglen,Ave,NaN,Brampton,Brampton W,"$699,500","$726,000",Semi-Detac,Bungalow,2,1,2,N,1,None,Cen,Gas,3/16/2021,3/17/2021,"EXP REALTY,",ROYALSTARRE,W5153813
4,5,Sld,NaN,121,Richwood,Cres,NaN,Brampton,Brampton W,"$699,900","$731,000",Att/Row/Tw,2-Storey,3,0,3,N,1,Built-,Cen,Gas,2/21/2021,2/26/2021,RE/MAX REAL,ROYAL LEPAGE,W5123258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1188,1189,Sld,NaN,16,Ravenscrof,Circ,NaN,Brampton,Brampton N,"$1,799,800","$1,880,000",Detached,2-Storey,5,0,5,Y,1,Attach,Cen,Gas,02/04/2021,02/08/2021,RE/MAX REAL,CENTURY 21 PR,W5103826
1189,1190,Sld,NaN,5,Dokkum,Cres,NaN,Brampton,Brampton N,"$1,799,900","$1,795,000",Detached,2-Storey,4,0,5,Y,1,Attach,Cen,Gas,4/13/2021,4/18/2021,RIGHT AT HO,CENTURY 21 PE,W5192314
1190,1191,Sld,NaN,95,Martin Byr,Dr,NaN,Brampton,Brampton W,"$1,800,000","$1,850,000",Detached,2-Storey,5,1,5,Y,1,Attach,Cen,Gas,3/26/2021,4/13/2021,TRIPLE NET,RE/MAX GOLD R,W5168825
1191,1192,Sld,NaN,52,Boyce,Cres,NaN,Brampton,Brampton W,"$1,849,000","$1,850,000",Detached,2-Storey,5,0,5,Y,1,Attach,Cen,Gas,2/15/2021,2/15/2021,RE/MAX REAL,RE/MAX REAL E,W5117936


In [10]:
features_df = sold_homes_df.drop(['List Price', 'Sold Price', 'Contract Date', 'Sold Date', 'List Brokerage', 'Co op Brokerage'], 1, inplace=False)
features_df

,#,LSC,EC,St#,Street Name,Abbr,Dir,Municipality,Community,Type,Style,Br,Additional,Wr,Fam,Kit,Garage Type,A/C,Heat,MLS#
0,1,Sld,NaN,38,Scarsdale,Crt,NaN,Brampton,Brampton W,Att/Row/Tw,2-Storey,3,0,2,N,1,Built-,Cen,Gas,W5153139
1,2,Sld,NaN,55,Corby,Cres,NaN,Brampton,Brampton W,Semi-Detac,Bungalow,3,1,2,N,1,None,Cen,Oil,W5125388
2,3,Sld,NaN,7,Manett,Cres,NaN,Brampton,Brampton W,Att/Row/Tw,2-Storey,3,1,3,N,1,Attach,Cen,Gas,W5091089
3,4,Sld,NaN,76,Fairglen,Ave,NaN,Brampton,Brampton W,Semi-Detac,Bungalow,2,1,2,N,1,None,Cen,Gas,W5153813
4,5,Sld,NaN,121,Richwood,Cres,NaN,Brampton,Brampton W,Att/Row/Tw,2-Storey,3,0,3,N,1,Built-,Cen,Gas,W5123258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1188,1189,Sld,NaN,16,Ravenscrof,Circ,NaN,Brampton,Brampton N,Detached,2-Storey,5,0,5,Y,1,Attach,Cen,Gas,W5103826
1189,1190,Sld,NaN,5,Dokkum,Cres,NaN,Brampton,Brampton N,Detached,2-Storey,4,0,5,Y,1,Attach,Cen,Gas,W5192314
1190,1191,Sld,NaN,95,Martin Byr,Dr,NaN,Brampton,Brampton W,Detached,2-Storey,5,1,5,Y,1,Attach,Cen,Gas,W5168825
1191,1192,Sld,NaN,52,Boyce,Cres,NaN,Brampton,Brampton W,Detached,2-Storey,5,0,5,Y,1,Attach,Cen,Gas,W5117936


In [11]:
prices_df = sold_homes_df.drop(['LSC', 'EC', 'St#', 'Street Name', 'Abbr', 'Dir', 'Municipality', 'Community', 'Type', 'Style', 'Br', 'Additional', 'Wr', 'Fam', 'Kit', 'Garage Type', 'A/C', 'Heat'], 1, inplace=False)
prices_df

,#,List Price,Sold Price,Contract Date,Sold Date,List Brokerage,Co op Brokerage,MLS#
0,1,"$599,000","$681,000",3/16/2021,3/23/2021,RE/MAX WEST,RE/MAX REALTY,W5153139
1,2,"$599,900","$767,000",2/23/2021,2/26/2021,RE/MAX REAL,HOMELIFE/MIRA,W5125388
2,3,"$699,000","$818,500",1/21/2021,1/26/2021,RE/MAX REAL,HOMELIFE/MIRA,W5091089
3,4,"$699,500","$726,000",3/16/2021,3/17/2021,"EXP REALTY,",ROYALSTARRE,W5153813
4,5,"$699,900","$731,000",2/21/2021,2/26/2021,RE/MAX REAL,ROYAL LEPAGE,W5123258
...,...,...,...,...,...,...,...,...
1188,1189,"$1,799,800","$1,880,000",02/04/2021,02/08/2021,RE/MAX REAL,CENTURY 21 PR,W5103826
1189,1190,"$1,799,900","$1,795,000",4/13/2021,4/18/2021,RIGHT AT HO,CENTURY 21 PE,W5192314
1190,1191,"$1,800,000","$1,850,000",3/26/2021,4/13/2021,TRIPLE NET,RE/MAX GOLD R,W5168825
1191,1192,"$1,849,000","$1,850,000",2/15/2021,2/15/2021,RE/MAX REAL,RE/MAX REAL E,W5117936


# Create Database

In [12]:
# Create the engine
engine = create_engine("sqlite:///sold_homes.db", echo=False)
sqlite_connection = engine.connect()

# Reflect existing db into a new model
Base = automap_base()
# Refelct the tables
Base.prepare(engine, reflect=True)

In [13]:
# Write features DataFrame to a SQLite table
features_df.to_sql('home_features', con=engine, if_exists='replace')

# Write prices DataFrame to a SQLite table
prices_df.to_sql('home_prices', con=engine, if_exists='replace')

In [14]:
sqlite_connection.close()